In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Concatenate, Reshape, Dropout
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [3]:
N, P = 3, 25

In [ ]:
def modelo_gurobi(N: int, P: int, T: int, Q: int, w: list, K: list, c: list, beta: list):
    if len(K) != N:
        raise ValueError(f"La longitud de K ({len(K)}) no coincide con N ({N})")
    if len(c) != N:
        raise ValueError(f"La longitud de c ({len(c)}) no coincide con N ({N})")
    if len(beta) != P:
        raise ValueError(f"La longitud de beta ({len(beta)}) no coincide con P ({P})")

    modelo_gurobi = gp.Model("asignacion_quirófanos")

    x = modelo_gurobi.addVars(P, N, vtype=GRB.BINARY, name="x")
    y = modelo_gurobi.addVars(N, vtype=GRB.BINARY, name="y")
    z = modelo_gurobi.addVars(P, vtype=GRB.BINARY, name="z")
    t = modelo_gurobi.addVars(N, vtype=GRB.CONTINUOUS, lb=0.0, name="t")

    modelo_gurobi.setObjective(
        gp.quicksum(K[j] * y[j] + c[j] * t[j] for j in range(N)) + gp.quicksum(beta[i] * z[i] for i in range(P)),
        GRB.MINIMIZE
    )

    # Restricciones
    for i in range(P):
        modelo_gurobi.addConstr(gp.quicksum(x[i, j] for j in range(N)) + z[i] == 1)
    for j in range(N):
        modelo_gurobi.addConstr(gp.quicksum(x[i, j] for i in range(P)) <= P * y[j])
    for j in range(N):
        modelo_gurobi.addConstr(gp.quicksum(w[i] * x[i, j] for i in range(P)) <= T + t[j])
    modelo_gurobi.addConstr(gp.quicksum(t[j] for j in range(N)) <= Q)

    # agregar restriccion de que z_i <= z_i+1
    for i in range(P - 1):
        modelo_gurobi.addConstr(z[i] <= z[i + 1])

    modelo_gurobi.Params.OutputFlag = 0

    modelo_gurobi.Params.TimeLimit = 5 * 60
    modelo_gurobi.update()
    modelo_gurobi.optimize()

    if modelo_gurobi.Status == GRB.OPTIMAL or modelo_gurobi.Status == GRB.SUBOPTIMAL or modelo_gurobi.Status == GRB.TIME_LIMIT:
        x_values = np.array([[x[i, j].x for j in range(N)] for i in range(P)])
        y_values = np.array([y[j].x for j in range(N)])
        z_values = np.array([z[i].x for i in range(P)])
        t_values = np.array([t[j].x for j in range(N)])

        x_values = x_values.tolist()
        y_values = y_values.tolist()
        z_values = z_values.tolist()
        t_values = t_values.tolist()

        return x_values, y_values, z_values, t_values
    else:
        print(f"No se encontró una solución factible. Estado: {modelo_gurobi.Status}")
        return None, None, None, None


In [5]:
def generar_instancias_y_resolver(num_instancias, N, P):
    archivo_resultados = "resultados.csv"
    es_primera_instancia = True

    for _ in range(num_instancias):
        w = np.random.randint(4, 16, P).tolist()  # P elementos, en horas
        K = np.random.randint(400, 500, N).tolist()  # N elementos, en dólares
        c = np.random.randint(50, 100, N).tolist()  # N elementos, en dólares
        beta = np.random.randint(500, 700, P).tolist()  # P elementos, en dólares
        T = 32 # 8 horas disponibles en cuartos de hora
        Q = 16 # 4 horas disponibles de sobretiempo en cuartos de hora

        x_values, y_values, z_values, t_values = modelo_gurobi(
            N, P, T, Q, w, K, c, beta
        )
        
        if x_values is not None:
            # Preparar los datos para guardar
            resultado = pd.DataFrame({
                "x_values": [x_values],
                "y_values": [y_values],
                "z_values": [z_values],
                "t_values": [t_values],
                "w": [w],
                "K": [K],
                "c": [c],
                "beta": [beta],
                "T": [T],
                "Q": [Q]
            })

            resultado.to_csv(archivo_resultados, mode="a", header=es_primera_instancia, index=False)
            
            es_primera_instancia = False
            print(_)


In [6]:
num_instancias = 10000
#generar_instancias_y_resolver(num_instancias, N, P)

In [7]:
# definir las entradas
input_N = Input(shape=(1,), name="input_N")
input_P = Input(shape=(1,), name="input_P")
input_T = Input(shape=(1,), name="input_T")
input_Q = Input(shape=(1,), name="input_Q")
input_w = Input(shape=(P,), name="input_w")  
input_K = Input(shape=(N,), name="input_K")  
input_c = Input(shape=(N,), name="input_c")  
input_beta = Input(shape=(P,), name="input_beta") 

input_scalars = Concatenate(name="inputs")([input_N, input_P, input_T, input_Q])
input_vectors = Concatenate(name="input_vectors")([input_w, input_K, input_c, input_beta])
all_inputs = Concatenate(name="all_inputs")([input_scalars, input_vectors])

# procesar las entradas con capas densas
dense = Dense(64, activation="relu")(all_inputs)
dense = Dense(64, activation="relu")(dense)

# salida X: matriz 25x3 (P x N)
output_x = Dense(25 * 3, activation="sigmoid", name="output_x")(dense)
output_x = Reshape((25, 3), name="reshape_x")(output_x)

# salida Y: vector 1x3 (1 x N)
output_y = Dense(3, activation="sigmoid", name="output_y")(dense)

# salida Z: vector 1x25 (1 x P)
output_z = Dense(25, activation="sigmoid", name="output_z")(dense)

# definir el modelo
model = Model(inputs=[input_N, input_P, input_T, input_Q, input_w, input_K, input_c, input_beta], 
              outputs=[output_x, output_y, output_z])

# compilar el modelo
model.compile(optimizer="adam", 
              loss={"reshape_x": "mse", "output_y": "mse", "output_z": "mse"},
              metrics={"reshape_x": "mae", "output_y": "mae", "output_z": "mae"})

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_N             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_P             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_T             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_Q             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_w             │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_K             │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_c             │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_beta          │ (None, 25)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inputs              │ (None, 4)         │          0 │ input_N[0][0],    │
│ (Concatenate)       │                   │            │ input_P[0][0],    │
│                     │                   │            │ input_T[0][0],    │
│                     │                   │            │ input_Q[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_vectors       │ (None, 56)        │          0 │ input_w[0][0],    │
│ (Concatenate)       │                   │            │ input_K[0][0],    │
│                     │                   │            │ input_c[0][0],    │
│                     │                   │            │ input_beta[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ all_inputs          │ (None, 60)        │          0 │ inputs[0][0],     │
│ (Concatenate)       │                   │            │ input_vectors[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      3,904 │ all_inputs[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      4,160 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_x (Dense)    │ (None, 75)        │      4,875 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_x (Reshape) │ (None, 25, 3)     │          0 │ output_x[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_y (Dense)    │ (None, 3)         │        195 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_z (Dense)    │ (None, 25)        │      1,625 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 14,759 (57.65 KB)

 Trainable params: 14,759 (57.65 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# pasar los datos del csv a una lista resultados
data = pd.read_csv("resultados.csv")
resultados = data.values.tolist()

In [9]:
param_N = [3 for i in range(num_instancias)]
param_P = [25 for i in range(num_instancias)]
param_w = [resultados[i][4] for i in range(num_instancias)]
param_w = np.array([eval(param_w[i]) for i in range(num_instancias)])

param_K = [resultados[i][5] for i in range(num_instancias)]
param_K = np.array([eval(param_K[i]) for i in range(num_instancias)])

param_c = [resultados[i][6] for i in range(num_instancias)]
param_c = np.array([eval(param_c[i]) for i in range(num_instancias)])

param_beta = [resultados[i][7] for i in range(num_instancias)]
param_beta = np.array([eval(param_beta[i]) for i in range(num_instancias)])

param_T = [resultados[i][8] for i in range(num_instancias)]
param_Q = [resultados[i][9] for i in range(num_instancias)]

x_values = [resultados[i][0] for i in range(num_instancias)]
x_values = np.array([eval(x_values[i]) for i in range(num_instancias)])

y_values = [resultados[i][1] for i in range(num_instancias)]
y_values = np.array([eval(y_values[i]) for i in range(num_instancias)])

z_values = [resultados[i][2] for i in range(num_instancias)]
z_values = np.array([eval(z_values[i]) for i in range(num_instancias)])

t_values = [resultados[i][3] for i in range(num_instancias)]
t_values = np.array([eval(t_values[i]) for i in range(num_instancias)])


In [10]:
X_N_train, X_N_test = train_test_split(param_N, test_size=0.2, random_state=42)
X_P_train, X_P_test = train_test_split(param_P, test_size=0.2, random_state=42)
X_T_train, X_T_test = train_test_split(param_T, test_size=0.2, random_state=42)
X_Q_train, X_Q_test = train_test_split(param_Q, test_size=0.2, random_state=42)
X_w_train, X_w_test = train_test_split(param_w, test_size=0.2, random_state=42)
X_K_train, X_K_test = train_test_split(param_K, test_size=0.2, random_state=42)
X_c_train, X_c_test = train_test_split(param_c, test_size=0.2, random_state=42)
X_beta_train, X_beta_test = train_test_split(param_beta, test_size=0.2, random_state=42)

x_train, x_test = train_test_split(x_values, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y_values, test_size=0.2, random_state=42)
z_train, z_test = train_test_split(z_values, test_size=0.2, random_state=42)
t_train, t_test = train_test_split(t_values, test_size=0.2, random_state=42)

In [11]:
X_N_train = np.array(X_N_train)
X_P_train = np.array(X_P_train)
X_T_train = np.array(X_T_train)
X_Q_train = np.array(X_Q_train)
X_w_train = np.array(X_w_train)
X_K_train = np.array(X_K_train)
X_c_train = np.array(X_c_train)
X_beta_train = np.array(X_beta_train)

X_N_test = np.array(X_N_test)
X_P_test = np.array(X_P_test)
X_T_test = np.array(X_T_test)
X_Q_test = np.array(X_Q_test)
X_w_test = np.array(X_w_test)
X_K_test = np.array(X_K_test)
X_c_test = np.array(X_c_test)
X_beta_test = np.array(X_beta_test)

x_train = np.array(x_train)
y_train = np.array(y_train)
z_train = np.array(z_train)
t_train = np.array(t_train)

x_test = np.array(x_test)
y_test = np.array(y_test)
z_test = np.array(z_test)
t_test = np.array(t_test)

# hacerle round a las salidas
x_train = np.round(x_train)
y_train = np.round(y_train)
z_train = np.round(z_train)

x_test = np.round(x_test)
y_test = np.round(y_test)
z_test = np.round(z_test)

# ver todas las shapes
print(X_N_train.shape)
print(X_P_train.shape)
print(X_T_train.shape)
print(X_Q_train.shape)
print(X_w_train.shape)
print(X_K_train.shape)
print(X_c_train.shape)
print(X_beta_train.shape)
print()
print(x_train.shape)
print(y_train.shape)
print(z_train.shape)
print(t_train.shape)



(8000,)
(8000,)
(8000,)
(8000,)
(8000, 25)
(8000, 3)
(8000, 3)
(8000, 25)

(8000, 25, 3)
(8000, 3)
(8000, 25)
(8000, 3)


In [12]:
print(f"X_N_train dtype: {X_N_train.dtype}")
print(f"X_P_train dtype: {X_P_train.dtype}")
print(f"X_T_train dtype: {X_T_train.dtype}")
print(f"X_Q_train dtype: {X_Q_train.dtype}")
print(f"X_w_train dtype: {X_w_train.dtype}")
print(f"X_K_train dtype: {X_K_train.dtype}")
print(f"X_c_train dtype: {X_c_train.dtype}")
print(f"X_beta_train dtype: {X_beta_train.dtype}")

print(f"x_train dtype: {x_train.dtype}")
print(f"y_train dtype: {y_train.dtype}")
print(f"z_train dtype: {z_train.dtype}")
print(f"t_train dtype: {t_train.dtype}")


X_N_train dtype: int64
X_P_train dtype: int64
X_T_train dtype: int64
X_Q_train dtype: int64
X_w_train dtype: int64
X_K_train dtype: int64
X_c_train dtype: int64
X_beta_train dtype: int64
x_train dtype: float64
y_train dtype: float64
z_train dtype: float64
t_train dtype: float64


In [13]:
model.fit(
    [X_N_train, X_P_train, X_T_train, X_Q_train, X_w_train, X_K_train, X_c_train, X_beta_train],
    {"reshape_x": x_train, "output_y": y_train, "output_z": z_train},
    validation_data=(
        [X_N_test, X_P_test, X_T_test, X_Q_test, X_w_test, X_K_test, X_c_test, X_beta_test],
        {"reshape_x": x_test, "output_y": y_test, "output_z": z_test}
    ),
    epochs=50,
    batch_size=8
)


Epoch 1/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.4812 - output_y_loss: 0.6667 - output_y_mae: 0.6667 - output_z_loss: 0.3855 - output_z_mae: 0.3855 - reshape_x_loss: 0.4290 - reshape_x_mae: 0.4292 - val_loss: 1.4344 - val_output_y_loss: 0.6667 - val_output_y_mae: 0.6667 - val_output_z_loss: 0.3611 - val_output_z_mae: 0.3611 - val_reshape_x_loss: 0.4067 - val_reshape_x_mae: 0.4067
Epoch 2/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.4360 - output_y_loss: 0.6667 - output_y_mae: 0.6667 - output_z_loss: 0.3629 - output_z_mae: 0.3629 - reshape_x_loss: 0.4065 - reshape_x_mae: 0.4065 - val_loss: 1.4344 - val_output_y_loss: 0.6667 - val_output_y_mae: 0.6667 - val_output_z_loss: 0.3611 - val_output_z_mae: 0.3611 - val_reshape_x_loss: 0.4067 - val_reshape_x_mae: 0.4067
Epoch 3/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.4365 - output_y_loss: 0.6667 - output_y_mae: 0.6667 - output_z_loss: 0.3629 - output_z_mae: 0.3629 - reshape_x_loss: 0.4069 - reshape_x_mae:

In [ ]:
# guardar el modelo
#model.save("modelo_quirofanos.keras")

In [15]:
# cargar el modelo
model = tf.keras.models.load_model("modelo_quirofanos.keras")

In [16]:
# probar el conjunto de prueba
model.evaluate(
    [X_N_test, X_P_test, X_T_test, X_Q_test, X_w_test, X_K_test, X_c_test, X_beta_test],
    {"reshape_x": x_test, "output_y": y_test, "output_z": z_test}
)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.2855 - output_y_loss: 0.6667 - output_y_mae: 0.6667 - output_z_loss: 0.2805 - output_z_mae: 0.2805 - reshape_x_loss: 0.3384 - reshape_x_mae: 0.3384


[1.2862132787704468,
 0.3386177718639374,
 0.6666667461395264,
 0.28113096952438354,
 0.6666666865348816,
 0.2810799777507782,
 0.33846667408943176]

In [17]:
# predecir para algun valor de entrada
N = 3
P = 25
T = 32
Q = 16
w = np.random.randint(4, 16, P).tolist()  # P elementos, en horas
K = np.random.randint(400, 500, N).tolist()  # N elementos, en dolares
c = np.random.randint(50, 100, N).tolist()  # N elementos, en dolares
beta = np.random.randint(500, 700, P).tolist()  # P elementos, en dolares

x_pred, y_pred, z_pred = model.predict(
    [np.array([N]), np.array([P]), np.array([T]), np.array([Q]), np.array([w]), np.array([K]), np.array([c]), np.array([beta])]
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step


In [18]:
print("x_pred:")
print(np.round(x_pred[0]))

x_pred:
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 1.]
 [1. 1. 1.]
 [1. 0. 1.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [1. 0. 1.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]


In [19]:
print("y_pred:")
print(y_pred)

y_pred:
[[1. 0. 0.]]


In [20]:
print("z_pred:")
print(np.round(z_pred[0]))

z_pred:
[0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0.
 0.]


In [21]:
# encontrar la solución óptima
x_values, y_values, z_values, t_values = modelo_gurobi(N, P, T, Q, w, K, c, beta)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-17


In [22]:
print("x_values:")
for i in range(P):
    print(x_values[i])

x_values:
[-0.0, -0.0, 1.0]
[-0.0, -0.0, 1.0]
[-0.0, -0.0, 1.0]
[-0.0, 1.0, -0.0]
[1.0, -0.0, -0.0]
[-0.0, -0.0, 1.0]
[-0.0, 1.0, -0.0]
[1.0, -0.0, -0.0]
[1.0, -0.0, -0.0]
[-0.0, 1.0, -0.0]
[-0.0, -0.0, 1.0]
[-0.0, -0.0, -0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]


In [23]:
print("y_values:")
print(y_values)

y_values:
[1.0, 1.0, 1.0]


In [24]:
print("z_values:")
print(z_values)

z_values:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [25]:
print("t_values:")
print(t_values)

t_values:
[-0.0, -0.0, -0.0]
